In [1]:
import pandas as pd 
import numpy as np
import copy
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16.0, 9.0)

In [2]:
!pip install xlrd

## 1. Upload original data

In [3]:
#upload price data
price1 = pd.read_excel('price1.xls')
price2 = pd.read_excel('price2.xls')
price3 = pd.read_excel('price3.xls')
price4 = pd.read_excel('price4.xls')
price5 = pd.read_excel('price5.xls')
price6 = pd.read_excel('price6.xls')
price7 = pd.read_excel('price7.xls')
price=pd.concat([price1,price2,price3,price4,price5,price6,price7], axis=0)
price=price.drop(columns=['复权价1(元)_AdjClpr1',	'复权价2(元)_AdjClpr2','成交量_Trdvol','成交金额_Trdsum'])
price.columns=['stkcd','ind','date','price']
output_csv_file = 'price.csv'
price.to_csv(output_csv_file, index=False)


In [4]:
price.head()


,stkcd,ind,date,price
0,1,J,2000-01-28,18.53
1,1,J,2000-02-29,18.32
2,1,J,2000-03-31,18.37
3,1,J,2000-04-28,19.05
4,1,J,2000-05-31,18.00


In [5]:
#upload stock quantities data
sharequa1 = pd.read_excel('sharequa1.xls')
sharequa2 = pd.read_excel('sharequa2.xls')
sharequa3 = pd.read_excel('sharequa3.xls')
sharequa4 = pd.read_excel('sharequa4.xls')
sharequa5 = pd.read_excel('sharequa5.xls')
sharequa6 = pd.read_excel('sharequa6.xls')
sharequa7 = pd.read_excel('sharequa7.xls')
sharequa=pd.concat([sharequa1,sharequa2,sharequa3,sharequa4,sharequa5,sharequa6,sharequa7], axis=0)
sharequa=sharequa.drop(columns=['证监会行业门类代码_Csrciccd1','流通股_Trdshr'])
sharequa.columns=['stkcd','date','quantity']
output_csv_file = 'sharequa.csv'
sharequa.to_csv(output_csv_file, index=False)

In [6]:
sharequa.head()


,stkcd,date,quantity
0,1,2000-01-28,1.551847e+09
1,1,2000-02-29,1.551847e+09
2,1,2000-03-31,1.551847e+09
3,1,2000-04-28,1.551847e+09
4,1,2000-05-31,1.551847e+09


In [7]:
#upload risk free return data
rf1 = pd.read_excel('rf1.xls')
rf2 = pd.read_excel('rf2.xls')
rf3 = pd.read_excel('rf3.xls')
rf4 = pd.read_excel('rf4.xls')
rf5 = pd.read_excel('rf5.xls')
rf6 = pd.read_excel('rf6.xls')
rf7 = pd.read_excel('rf7.xls')
rf=pd.concat([rf1,rf2,rf3,rf4,rf5,rf6,rf7], axis=0)
rf=rf.drop(columns=['证监会行业门类代码_Csrciccd1'])
rf.columns=['stkcd','date','rf']
output_csv_file = 'rf.csv'
rf.to_csv(output_csv_file, index=False)


In [8]:
rf.head()

,stkcd,date,rf
0,1,2001-01-19,0.00165
1,1,2001-02-28,0.00165
2,1,2001-03-30,0.00165
3,1,2001-04-30,0.00165
4,1,2001-05-31,0.00165


In [9]:
#upload bookvalue per share data 
value1 = pd.read_excel('value1.xls')
value2 = pd.read_excel('value2.xls')
value3 = pd.read_excel('value3.xls')
value4 = pd.read_excel('value4.xls')
value5 = pd.read_excel('value5.xls')
value6 = pd.read_excel('value6.xls')
value7 = pd.read_excel('value7.xls')
value=pd.concat([value1,value2,value3,value4,value5,value6,value7], axis=0)
value=value.drop(columns=['证监会行业门类代码_Csrciccd1','调整后每股净资产(元/股)_NAPSadj'])
value.columns=['stkcd','date','bookvalue_share']
output_csv_file = 'value.csv'
value.to_csv(output_csv_file, index=False)

In [10]:
value.head()

,stkcd,date,bookvalue_share
0,1,2000-01-28,2.52
1,1,2000-02-29,2.52
2,1,2000-03-31,2.52
3,1,2000-04-28,1.87
4,1,2000-05-31,1.87


## 2.Data processing in separate table

In [11]:
#process data in table rf
rf['time_month'] = rf['date'].dt.to_period('M')
rf = rf[rf['stkcd'] == 1]
rf=rf.drop(columns=['date','stkcd'])
rf.dropna(inplace=True)

In [12]:
rf.head()

,rf,time_month
0,0.00165,2001-01
1,0.00165,2001-02
2,0.00165,2001-03
3,0.00165,2001-04
4,0.00165,2001-05


In [13]:
#process data in price and sharequa
mkt=pd.merge(price,sharequa,on=['stkcd','date'])
mkt['time_month']=mkt['date'].dt.to_period('M')
mkt['year']=mkt['date'].dt.year
mkt['price']=abs(mkt['price'])
mkt = mkt[mkt['ind'] != 'J']

In [14]:
mkt.head()

,stkcd,ind,date,price,quantity,time_month,year
72,2,K,2000-01-28,10.55,423785259.0,2000-01,2000
73,2,K,2000-02-29,14.83,509216805.0,2000-02,2000
74,2,K,2000-03-31,12.69,509216805.0,2000-03,2000
75,2,K,2000-04-28,12.48,509216805.0,2000-04,2000
76,2,K,2000-05-31,11.89,509216805.0,2000-05,2000


In [15]:
#process data in value
value['year']=value['date'].dt.year

In [16]:
value.head()

,stkcd,date,bookvalue_share,year
0,1,2000-01-28,2.52,2000
1,1,2000-02-29,2.52,2000
2,1,2000-03-31,2.52,2000
3,1,2000-04-28,1.87,2000
4,1,2000-05-31,1.87,2000


### 3. Figure out excess return 

In [17]:
#calculate stock return
ret=copy.deepcopy(mkt.loc[:,['stkcd','price','time_month']])
ret.dropna(inplace=True)
ret['ret']=ret.groupby('stkcd')['price'].apply(lambda x : x/x.shift() - 1)
ret.drop(columns=['price'],inplace=True)
ret.dropna(inplace=True)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_10140\151536120.py:4: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  ret['ret']=ret.groupby('stkcd')['price'].apply(lambda x : x/x.shift() - 1)


In [18]:
ret.head()

,stkcd,time_month,ret
73,2,2000-02,0.405687
74,2,2000-03,-0.144302
75,2,2000-04,-0.016548
76,2,2000-05,-0.047276
77,2,2000-06,0.136249


In [19]:
rf.head()

,rf,time_month
0,0.00165,2001-01
1,0.00165,2001-02
2,0.00165,2001-03
3,0.00165,2001-04
4,0.00165,2001-05


In [20]:
#calculate excess return
excessret = pd.merge(ret,rf,on=['time_month'])
excessret.sort_values(['stkcd','time_month'],inplace=True)
excessret['excessr']=excessret['ret']-excessret['rf']

In [21]:
excessret.head()

,stkcd,time_month,ret,rf,excessr
10,2,2001-01,0.067191,0.00165,0.065541
1103,2,2001-02,-0.077696,0.00165,-0.079346
2203,2,2001-03,0.098765,0.00165,0.097115
3326,2,2001-04,-0.036352,0.00165,-0.038002
4453,2,2001-05,-0.026063,0.00165,-0.027713


### 4. Portfolio construction (Small, big, low, medium, high)

#### 4.1 figure out BM ratio

In [22]:
#deal with book value elements
bookvalue = copy.deepcopy(value)
bookvalue = bookvalue.groupby(['stkcd','year']).last()
bookvalue.reset_index(inplace=True)
bookvalue.dropna(inplace=True)
bookvalue.drop(columns=['date'],inplace=True)


In [23]:
bookvalue.head()

,stkcd,year,bookvalue_share
0,1,2000,1.98
1,1,2001,2.55
2,1,2002,2.09
3,1,2003,2.05
4,1,2004,2.25


In [24]:
#deal with market value elements
marketvalue= copy.deepcopy(mkt.loc[:,['stkcd','ind','quantity','price','time_month','year']])
marketvalue = marketvalue.groupby(['stkcd','year']).last()
marketvalue.reset_index(inplace=True)
marketvalue.dropna(inplace=True)

In [25]:
marketvalue.head()

,stkcd,year,ind,quantity,price,time_month
0,2,2000,K,5.092168e+08,13.99,2000-12
1,2,2001,K,5.092168e+08,13.35,2001-12
2,2,2002,K,5.092196e+08,9.65,2002-12
3,2,2003,K,1.152339e+09,6.54,2003-12
4,2,2004,K,1.908362e+09,5.26,2004-12


In [26]:
#merge two kinds of values
bothvalue = pd.merge(bookvalue,marketvalue,on=['stkcd','year'])

In [27]:
#calculate market value
bothvalue['mktv_total']=bothvalue['price']*bothvalue['quantity']
#calculate book value
bothvalue['bkv_total']=bothvalue['bookvalue_share']*bothvalue['quantity']
#calculate BM ratio
bothvalue['bm']=bothvalue['bkv_total']/bothvalue['mktv_total']
bothvalue.drop(columns=['bookvalue_share','quantity','price','time_month','mktv_total','bkv_total','ind'],inplace=True)

In [28]:
bothvalue.head()

,stkcd,year,bm
0,2,2000,0.320944
1,2,2001,0.361798
2,2,2002,0.544041
3,2,2003,0.484709
4,2,2004,0.477186


#### 4.2 Data process and table merge 
main purpose: preparing for the calculation of the other two factors

In [29]:
#independent lag one period
bothvalue.rename(columns={'year':'bm_year'},inplace=True)
excessret['year']=excessret['time_month'].dt.year
excessret['month']=excessret['time_month'].dt.month
excessret['bm_year']=excessret['year']-1
month_index=excessret['month'].isin([1,2,3,4,5,6])
#why lag one period again?
#because author Assume that the data on January-June rates of return match the previous year's financial information
excessret.loc[month_index,'bm_year']=excessret.loc[month_index,'bm_year']-1
#merge excess return and bm tables
reg1=pd.merge(excessret,bothvalue,on=['stkcd','bm_year'])


In [30]:
excessret.head()

,stkcd,time_month,ret,rf,excessr,year,month,bm_year
10,2,2001-01,0.067191,0.00165,0.065541,2001,1,1999
1103,2,2001-02,-0.077696,0.00165,-0.079346,2001,2,1999
2203,2,2001-03,0.098765,0.00165,0.097115,2001,3,1999
3326,2,2001-04,-0.036352,0.00165,-0.038002,2001,4,1999
4453,2,2001-05,-0.026063,0.00165,-0.027713,2001,5,1999


In [31]:
reg1.head()

,stkcd,time_month,ret,rf,excessr,year,month,bm_year,bm
0,2,2001-07,-0.066172,0.00165,-0.067822,2001,7,2000,0.320944
1,2,2001-08,0.055676,0.00165,0.054026,2001,8,2000,0.320944
2,2,2001-09,0.011644,0.00165,0.009994,2001,9,2000,0.320944
3,2,2001-10,-0.092079,0.00165,-0.093729,2001,10,2000,0.320944
4,2,2001-11,0.006711,0.00165,0.005061,2001,11,2000,0.320944


In [32]:
#figure out market value（for weight)
marketcap=copy.deepcopy(mkt.loc[:,['stkcd','quantity','price','time_month','year']])
marketcap['marketcap']=marketcap['price']*marketcap['quantity']
marketcap.rename(columns={'time_month':'mkt_month'},inplace=True)
#merge the log1 with market value
reg1['mkt_date'] = reg1['year']
month_index2 = reg1['month'].isin([1,2,3,4,5,6])
reg1.loc[month_index2,'mkt_date'] = reg1.loc[month_index2,'mkt_date'] - 1
reg1['mkt_date'] = pd.to_datetime(reg1['mkt_date'].astype('str'),format='%Y') +pd.DateOffset(months=5)
reg1['mkt_month'] = reg1['mkt_date'].dt.to_period('M')
# why lag one period?
# Because author Assume that the data on January-June rates of return match the previous year's financial informatiore
reg1 = pd.merge(reg1,marketcap,on=['stkcd','mkt_month'])
reg1.drop(columns=['mkt_date'],inplace=True)

In [33]:
reg1.head()

,stkcd,time_month,ret,rf,excessr,year_x,month,bm_year,bm,mkt_month,quantity,price,year_y,marketcap
0,2,2001-07,-0.066172,0.00165,-0.067822,2001,7,2000,0.320944,2001-06,509216805.0,14.81,2001,7.541501e+09
1,2,2001-08,0.055676,0.00165,0.054026,2001,8,2000,0.320944,2001-06,509216805.0,14.81,2001,7.541501e+09
2,2,2001-09,0.011644,0.00165,0.009994,2001,9,2000,0.320944,2001-06,509216805.0,14.81,2001,7.541501e+09
3,2,2001-10,-0.092079,0.00165,-0.093729,2001,10,2000,0.320944,2001-06,509216805.0,14.81,2001,7.541501e+09
4,2,2001-11,0.006711,0.00165,0.005061,2001,11,2000,0.320944,2001-06,509216805.0,14.81,2001,7.541501e+09


#### 4.3 Portfolio construction 

In [34]:
reg1copy=copy.deepcopy(reg1)

In [35]:
reg1.drop(columns=['year_x','bm_year','month','price','quantity','year_y'],inplace= True)

In [36]:
#Calculation of the 30 percent and 70 percent quartiles of bm
bm_percent = pd.DataFrame()
bm_percent['bm1'] = reg1.groupby(['time_month'])['bm'].quantile(0.3) 
bm_percent['bm2'] = reg1.groupby(['time_month'])['bm'].quantile(0.7)
#Calculation of the 50 percent quartiles of market capital
cap_percent=pd.DataFrame()
cap_percent['cap'] = reg1.groupby(['time_month'])['marketcap'].quantile(0.5) 
reg1=pd.merge(reg1,bm_percent,on=['time_month'])
reg1=pd.merge(reg1,cap_percent,on=['time_month'])

In [37]:
reg1. dropna(inplace=True)
reg1.drop(columns=['mkt_month'],inplace=True)

In [38]:
reg1.head()

,stkcd,time_month,ret,rf,excessr,bm,marketcap,bm1,bm2,cap
0,2,2001-07,-0.066172,0.00165,-0.067822,0.320944,7.541501e+09,0.130464,0.232823,3.442840e+09
1,5,2001-07,-0.212454,0.00165,-0.214104,0.253558,5.337257e+09,0.130464,0.232823,3.442840e+09
2,6,2001-07,-0.054519,0.00165,-0.056169,0.324754,3.535067e+09,0.130464,0.232823,3.442840e+09
3,8,2001-07,-0.261885,0.00165,-0.263535,0.030213,1.735270e+09,0.130464,0.232823,3.442840e+09
4,9,2001-07,-0.178620,0.00165,-0.180270,0.183989,7.085606e+09,0.130464,0.232823,3.442840e+09


In [39]:
#construct portfolios for different level of bm
bm_portfolios=dict()
selected_column= ['stkcd', 'time_month', 'ret', 'rf', 'excessr', 'bm', 'marketcap']
bm_portfolios['BM1'] = reg1.query('bm <= bm1')[selected_column]
bm_portfolios['BM2'] = reg1.query('bm >= bm1 and bm <= bm2')[selected_column]
bm_portfolios['BM3'] = reg1.query('bm >= bm2')[selected_column]

In [40]:
for key in bm_portfolios.keys():
    bm_portfolios[key] = bm_portfolios[key][bm_portfolios[key]['bm']>=0]

In [41]:
#construct portfolios for different level of market capital
cap_portfolios=dict()
selected_column= ['stkcd', 'time_month', 'ret', 'rf', 'excessr', 'bm', 'marketcap']
cap_portfolios['CAP1'] = reg1.query('marketcap <= cap')[selected_column]
cap_portfolios['CAP2'] = reg1.query('marketcap >= cap')[selected_column]


In [42]:
#construct portfolios combining different market capital and bm
portfolios = dict()
for bm in bm_portfolios.keys():
    for cap in cap_portfolios.keys():
        portfolios[f'{bm}_{cap}'] = pd.merge(bm_portfolios[bm][['stkcd','time_month']],cap_portfolios[cap],on=['stkcd','time_month'])

In [43]:
#Calculate market capitalization weights for each portfolio
portfolios_weightr = dict()
for tb in portfolios.keys():
    temp = pd.DataFrame(portfolios[tb].groupby('time_month')['marketcap'].sum())
    temp.columns = ['marketcap_sum']
    portfolios[tb] = pd.merge(portfolios[tb],temp,on=['time_month'])
    portfolios[tb]['weight'] = portfolios[tb][
        'marketcap']/portfolios[tb]['marketcap_sum']
    portfolios[tb]['weighted_ret'] = portfolios[tb]['weight']*portfolios[tb]['ret']
    portfolios_weightr[tb] = portfolios[tb].groupby(['time_month'])[
        ['weighted_ret']].sum()

In [44]:
portfolios_weightr['BM1_CAP1'].head()

,weighted_ret
time_month,
2001-07,-0.143726
2001-08,-0.023984
2001-09,-0.068680
2001-10,-0.074385
2001-11,0.050267


In [45]:
portfolios_weightr_df = pd.DataFrame(np.hstack([table for table in portfolios_weightr.values()]))
portfolios_weightr_df.index = portfolios_weightr['BM1_CAP1'].index
portfolios_weightr_df.columns = portfolios_weightr.keys()

In [46]:
portfolios_weightr_df.head()

,BM1_CAP1,BM1_CAP2,BM2_CAP1,BM2_CAP2,BM3_CAP1,BM3_CAP2
time_month,,,,,,
2001-07,-0.143726,-0.136002,-0.137373,-0.133831,-0.171611,-0.140007
2001-08,-0.023984,-0.046494,-0.021304,-0.046586,-0.024247,-0.036457
2001-09,-0.068680,-0.063882,-0.066958,-0.058034,-0.079561,-0.038619
2001-10,-0.074385,-0.072584,-0.071946,-0.049890,-0.025153,-0.021317
2001-11,0.050267,0.025087,0.063935,0.046340,0.055789,0.037679


In [47]:
portfolios_weightr_df.rename(columns={'BM1_CAP1':'LS',
                                      'BM1_CAP2':'LB',
                                      'BM2_CAP1':'MS',
                                      'BM2_CAP2':'MB',
                                      'BM3_CAP1':'HS',
                                      'BM3_CAP2':'HB'},inplace=True)

### 5. Calculate SMB 、HML、RM-RF (Independence)

In [48]:
#SMB
SMB = (portfolios_weightr_df['LS'] + portfolios_weightr_df['MS'] + portfolios_weightr_df['HS']) / 3 - \
      (portfolios_weightr_df['LB'] + portfolios_weightr_df['MB'] + portfolios_weightr_df['HB']) / 3 

In [49]:
#HML
HML = (portfolios_weightr_df['HS'] + portfolios_weightr_df['HB'])/ 2 - \
      (portfolios_weightr_df['LS'] + portfolios_weightr_df['LB']) / 2

In [50]:
HML=pd.DataFrame(HML)
HML.columns=['HML']
SMB=pd.DataFrame(SMB)
SMB.columns=['SMB']

In [51]:
SMB

,SMB
time_month,
2001-07,-0.014290
2001-08,0.020001
2001-09,-0.018221
2001-10,-0.009231
2001-11,0.020295
...,...
2015-08,0.023811
2015-09,0.029536
2015-10,0.085839


In [52]:
HML

,HML
time_month,
2001-07,-0.015946
2001-08,0.004887
2001-09,0.007191
2001-10,0.050250
2001-11,0.009057
...,...
2015-08,0.013954
2015-09,-0.034584
2015-10,-0.053676


In [53]:
reg1copy.head()

,stkcd,time_month,ret,rf,excessr,year_x,month,bm_year,bm,mkt_month,quantity,price,year_y,marketcap
0,2,2001-07,-0.066172,0.00165,-0.067822,2001,7,2000,0.320944,2001-06,509216805.0,14.81,2001,7.541501e+09
1,2,2001-08,0.055676,0.00165,0.054026,2001,8,2000,0.320944,2001-06,509216805.0,14.81,2001,7.541501e+09
2,2,2001-09,0.011644,0.00165,0.009994,2001,9,2000,0.320944,2001-06,509216805.0,14.81,2001,7.541501e+09
3,2,2001-10,-0.092079,0.00165,-0.093729,2001,10,2000,0.320944,2001-06,509216805.0,14.81,2001,7.541501e+09
4,2,2001-11,0.006711,0.00165,0.005061,2001,11,2000,0.320944,2001-06,509216805.0,14.81,2001,7.541501e+09


In [54]:
#RM-RF
keep_cols = ['stkcd', 'time_month', 'ret', 'rf', 'marketcap']
reg2 = reg1copy[keep_cols]
temp1=pd.DataFrame(reg1copy.groupby(['time_month'])['marketcap'].sum())
temp.columns = ['marketcap_sum']
reg2 = pd.merge(reg2,temp,on=['time_month'])
reg2['weight'] = reg2['marketcap']/reg2['marketcap_sum']
reg2['weighted_ret'] = reg2['weight']*reg2['ret']
rm_rf = reg2.groupby(['time_month'])[['weighted_ret']].sum()
rm_rf.columns=['rm']
rm_rf=pd.merge(rm_rf,rf,on=['time_month'])
rm_rf['rm_rf']=rm_rf['rm']-rm_rf['rf']
rm_rf.set_index(['time_month'],inplace=True)
rm_rf.drop(columns=['rm','rf'],inplace=True)

In [55]:
rm_rf

,rm_rf
time_month,
2001-07,-0.648593
2001-08,-0.177721
2001-09,-0.272954
2001-10,-0.241529
2001-11,0.196788
...,...
2015-08,-0.406447
2015-09,-0.187340
2015-10,0.435152


In [56]:
#Combine all independent variables
independence0 = pd.merge(SMB,HML,on=['time_month'])
independence = pd.merge(independence0,rm_rf,on=['time_month'])


### 6. Calculate dependence

In [63]:
reg1copy.drop(columns=['year_x','year_y'],inplace=True)

In [66]:
reg1copy.head()

,stkcd,time_month,ret,rf,excessr,month,bm_year,bm,mkt_month,quantity,price,marketcap
0,2,2001-07,-0.066172,0.00165,-0.067822,7,2000,0.320944,2001-06,509216805.0,14.81,7.541501e+09
1,2,2001-08,0.055676,0.00165,0.054026,8,2000,0.320944,2001-06,509216805.0,14.81,7.541501e+09
2,2,2001-09,0.011644,0.00165,0.009994,9,2000,0.320944,2001-06,509216805.0,14.81,7.541501e+09
3,2,2001-10,-0.092079,0.00165,-0.093729,10,2000,0.320944,2001-06,509216805.0,14.81,7.541501e+09
4,2,2001-11,0.006711,0.00165,0.005061,11,2000,0.320944,2001-06,509216805.0,14.81,7.541501e+09


In [64]:
q = dict()
keys = ['q'+str(i) for i in range(1, 5)]
values = [0.2,0.4,0.6,0.8]
q.update(zip(keys,values))

In [67]:
quantile_df = pd.DataFrame()
for key, value in q.items():
    quantile_df[key] = reg1copy.groupby(['bm_year'])['bm'].quantile(value)

In [68]:
reg1copy = pd.merge(reg1copy, quantile_df, on='bm_year')

In [73]:
q = dict()
keys = ['Q'+str(i) for i in range(1, 5)]
values = [0.2,0.4,0.6,0.8]
q.update(zip(keys,values))
quantile_df = pd.DataFrame()
for key, value in q.items():
    quantile_df[key] = reg1copy.groupby(['mkt_month'])['marketcap'].quantile(value)
reg1copy = pd.merge(reg1copy, quantile_df, on='mkt_month')

In [69]:
#portfolios for different bm
portfolios_bm = dict()
keep_cols = ['stkcd', 'time_month', 'ret', 'rf', 'excessr', 'bm_year', 'bm',
       'mkt_month',  'marketcap' ]

portfolios_bm['bm1'] = reg1copy.loc[reg1copy['bm'] <= reg1copy['q1'],keep_cols].copy()
for i in range(2,5):
    idx = (reg1copy[f'q{i-1}'] <= reg1copy['bm']) & (reg1copy['bm'] <= reg1copy[f'q{i}'])
    portfolios_bm[f'bm{i}'] = reg1copy.loc[idx,keep_cols].copy()
portfolios_bm['bm5'] = reg1copy.loc[reg1copy['bm'] >= reg1copy['q4'],keep_cols].copy()

In [74]:
#portfolios for different market capital
portfolios_mkt = dict()
keep_cols = ['stkcd', 'time_month', 'ret', 'rf', 'excessr', 'bm_year', 'bm',
       'mkt_month',  'marketcap' ]

portfolios_mkt['mkt1'] = reg1copy.loc[reg1copy['marketcap'] <= reg1copy['Q1'],keep_cols].copy()
for i in range(2,5):
    idx = (reg1copy[f'Q{i-1}'] <= reg1copy['marketcap']) & (reg1copy['marketcap'] <= reg1copy[f'Q{i}'])
    portfolios_mkt[f'mkt{i}'] =reg1copy.loc[idx,keep_cols].copy()
portfolios_mkt['mkt5'] = reg1copy.loc[reg1copy['marketcap'] >= reg1copy['Q4'],keep_cols].copy()

In [75]:
#portfolios for dependence
portfolios_dpt = dict()
for bm in portfolios_bm.keys():
    for mkt in portfolios_mkt.keys():
        portfolios_dpt[f'{bm}_{mkt}'] = pd.merge(portfolios_bm[bm][['stkcd','time_month']],
                                            portfolios_mkt[mkt],
                                             on=['stkcd','time_month'])

In [80]:
portfolios_ret_mean = dict()
for k in portfolios_dpt.keys():
    portfolios_dpt[k]['weight'] = portfolios_dpt[k].groupby('time_month')['marketcap'].apply(lambda x: x / x.sum())
    portfolios_dpt[k]['weight_ret'] = portfolios_dpt[k]['ret'] * portfolios_dpt[k]['weight']
    portfolios_ret_mean[k] = portfolios_dpt[k].groupby(['time_month'])['weight_ret'].sum()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_10140\230186773.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  portfolios_dpt[k]['weight'] = portfolios_dpt[k].groupby('time_month')['marketcap'].apply(lambda x: x / x.sum())
C:\Users\lenovo\AppData\Local\Temp\ipykernel_10140\230186773.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, us

In [81]:
for k in portfolios_ret_mean.keys():
    portfolios_ret_mean[k] = pd.DataFrame(portfolios_ret_mean[k])
    portfolios_ret_mean[k].columns = ['weight_ret']
    portfolios_ret_mean[k] = pd.merge(portfolios_ret_mean[k],rf,on=['time_month'])
    portfolios_ret_mean[k]['weight_ex_ret'] = portfolios_ret_mean[k][
        'weight_ret'] - portfolios_ret_mean[k]['rf']

In [82]:
for key in portfolios_ret_mean.keys():
    portfolios_ret_mean[key].set_index(['time_month'],inplace=True)

In [83]:
#construct dependance variable
dependance = pd.DataFrame(np.hstack([pf[['weight_ex_ret']] for pf in portfolios_ret_mean.values()]))
dependance.index = portfolios_ret_mean['bm1_mkt1'].index
dependance.columns = portfolios_ret_mean.keys()

In [87]:
dependance.head()

,bm1_mkt1,bm1_mkt2,bm1_mkt3,bm1_mkt4,bm1_mkt5,bm2_mkt1,bm2_mkt2,bm2_mkt3,bm2_mkt4,bm2_mkt5,...,bm4_mkt1,bm4_mkt2,bm4_mkt3,bm4_mkt4,bm4_mkt5,bm5_mkt1,bm5_mkt2,bm5_mkt3,bm5_mkt4,bm5_mkt5
time_month,,,,,,,,,,,,,,,,,,,,,
2001-07,-0.140650,-0.140712,-0.148708,-0.122765,-0.142531,-0.145882,-0.149451,-0.126554,-0.149964,-0.116499,...,-0.148907,-0.140668,-0.134736,-0.149128,-0.131951,-0.237022,-0.180335,-0.147724,-0.147572,-0.145368
2001-08,-0.014234,-0.025091,-0.036927,-0.047635,-0.038246,0.017462,-0.025770,-0.035938,-0.043613,-0.066692,...,-0.045895,-0.019129,-0.038574,-0.041195,-0.031580,-0.022053,-0.015622,-0.046275,-0.055624,-0.038347
2001-09,-0.063857,-0.071170,-0.040629,-0.037287,-0.081498,-0.100799,-0.082482,-0.046697,-0.060096,-0.083605,...,-0.060489,-0.057569,-0.071867,-0.064402,-0.042925,-0.096620,-0.084478,-0.070801,-0.052971,-0.029324
2001-10,-0.085371,-0.081787,-0.085339,-0.119341,-0.062856,-0.068585,-0.072617,-0.082685,-0.075834,-0.054761,...,-0.059885,-0.056650,-0.049687,-0.037472,-0.028903,0.019431,-0.012839,-0.022605,-0.021243,-0.013409
2001-11,0.057110,0.038599,0.031710,0.024890,0.007065,0.073250,0.060746,0.041560,0.043956,0.026362,...,0.051718,0.064197,0.056126,0.049545,0.025995,0.049049,0.046792,0.040503,0.049919,0.042796


### 7. OLS estimation

In [130]:
dependance.to_csv('dependance.csv')
data=pd.merge(dependance,independence,on=['time_month'])
pd.merge(dependance,independence,on=['time_month']).to_csv('data1.csv')

In [91]:
import statsmodels.api as sm

In [129]:
data1.head()

,bm1_mkt1,bm1_mkt2,bm1_mkt3,bm1_mkt4,bm1_mkt5,bm2_mkt1,bm2_mkt2,bm2_mkt3,bm2_mkt4,bm2_mkt5,...,bm4_mkt4,bm4_mkt5,bm5_mkt1,bm5_mkt2,bm5_mkt3,bm5_mkt4,bm5_mkt5,SMB,HML,rm_rf
time_month,,,,,,,,,,,,,,,,,,,,,
2001-07,-0.140650,-0.140712,-0.148708,-0.122765,-0.142531,-0.145882,-0.149451,-0.126554,-0.149964,-0.116499,...,-0.149128,-0.131951,-0.237022,-0.180335,-0.147724,-0.147572,-0.145368,-0.014290,-0.015946,-0.648593
2001-08,-0.014234,-0.025091,-0.036927,-0.047635,-0.038246,0.017462,-0.025770,-0.035938,-0.043613,-0.066692,...,-0.041195,-0.031580,-0.022053,-0.015622,-0.046275,-0.055624,-0.038347,0.020001,0.004887,-0.177721
2001-09,-0.063857,-0.071170,-0.040629,-0.037287,-0.081498,-0.100799,-0.082482,-0.046697,-0.060096,-0.083605,...,-0.064402,-0.042925,-0.096620,-0.084478,-0.070801,-0.052971,-0.029324,-0.018221,0.007191,-0.272954
2001-10,-0.085371,-0.081787,-0.085339,-0.119341,-0.062856,-0.068585,-0.072617,-0.082685,-0.075834,-0.054761,...,-0.037472,-0.028903,0.019431,-0.012839,-0.022605,-0.021243,-0.013409,-0.009231,0.050250,-0.241529
2001-11,0.057110,0.038599,0.031710,0.024890,0.007065,0.073250,0.060746,0.041560,0.043956,0.026362,...,0.049545,0.025995,0.049049,0.046792,0.040503,0.049919,0.042796,0.020295,0.009057,0.196788


In [137]:
data.columns[0:25]

Index(['bm1_mkt1', 'bm1_mkt2', 'bm1_mkt3', 'bm1_mkt4', 'bm1_mkt5', 'bm2_mkt1',
       'bm2_mkt2', 'bm2_mkt3', 'bm2_mkt4', 'bm2_mkt5', 'bm3_mkt1', 'bm3_mkt2',
       'bm3_mkt3', 'bm3_mkt4', 'bm3_mkt5', 'bm4_mkt1', 'bm4_mkt2', 'bm4_mkt3',
       'bm4_mkt4', 'bm4_mkt5', 'bm5_mkt1', 'bm5_mkt2', 'bm5_mkt3', 'bm5_mkt4',
       'bm5_mkt5'],
      dtype='object')

In [138]:
#CAPM model (rm_rf as independent)
# Function to perform regression and return results
def perform_regression(y, X):
    model = sm.OLS(y, sm.add_constant(X)).fit()
    coefficients = model.params
    p_values = model.pvalues
    r_squared = model.rsquared
    return coefficients, p_values, r_squared

# Lists to store the regression results for each table
tables = {
    'table1': {'dependent': [], 'intercept': [], 'coefficients': [], 'r_squared': [], 'p_values': []},
    'table2': {'dependent': [], 'intercept': [], 'coefficients': [], 'r_squared': [], 'p_values': []},
    'table3': {'dependent': [], 'intercept': [], 'coefficients': [], 'r_squared': [], 'p_values': []}
}

# Table 1: Regression with 'rm_rf' as independent variable
for dep_var in data.columns[0:25]:
    coefficients, p_values, r_squared = perform_regression(data[dep_var], data['rm_rf'])
    tables['table1']['dependent'].append(dep_var)
    tables['table1']['intercept'].append(coefficients['const'])
    tables['table1']['coefficients'].append(coefficients['rm_rf'])
    tables['table1']['r_squared'].append(r_squared)
    tables['table1']['p_values'].append(p_values['rm_rf'])

# Table 2: Regression with 'SMB' and 'HML' as independent variables
for dep_var in data.columns[0:25]:
    coefficients, p_values, r_squared = perform_regression(data[dep_var], data[['SMB', 'HML']])
    tables['table2']['dependent'].append(dep_var)
    tables['table2']['intercept'].append(coefficients['const'])
    tables['table2']['coefficients'].append((coefficients['SMB'], coefficients['HML']))
    tables['table2']['r_squared'].append(r_squared)
    tables['table2']['p_values'].append((p_values['SMB'], p_values['HML']))

# Table 3: Regression with 'rm_rf', 'SMB' and 'HML' as independent variables
for dep_var in data.columns[0:25]:
    coefficients, p_values, r_squared = perform_regression(data[dep_var], data[['rm_rf', 'SMB', 'HML']])
    tables['table3']['dependent'].append(dep_var)
    tables['table3']['intercept'].append(coefficients['const'])
    tables['table3']['coefficients'].append((coefficients['rm_rf'], coefficients['SMB'], coefficients['HML']))
    tables['table3']['r_squared'].append(r_squared)
    tables['table3']['p_values'].append((p_values['rm_rf'], p_values['SMB'], p_values['HML']))

# Convert each table dictionary to DataFrame
table1_df = pd.DataFrame(tables['table1'])
table2_df = pd.DataFrame(tables['table2'])
table3_df = pd.DataFrame(tables['table3'])

table1_df.head(), table2_df.head(), table3_df.head()


(  dependent  intercept  coefficients  r_squared      p_values
 0  bm1_mkt1   0.009264      0.243913   0.600678  2.827735e-25
 1  bm1_mkt2   0.005611      0.240811   0.634126  1.579285e-27
 2  bm1_mkt3   0.002805      0.237024   0.660976  1.724784e-29
 3  bm1_mkt4   0.002269      0.222160   0.665310  8.048180e-30
 4  bm1_mkt5  -0.003072      0.195410   0.676339  1.105587e-30,
   dependent  intercept                              coefficients  r_squared  \
 0  bm1_mkt1  -0.009542  (1.7946679555802247, 0.5087628776670796)   0.469164   
 1  bm1_mkt2  -0.011895    (1.592418829641204, 0.413899847863858)   0.401300   
 2  bm1_mkt3  -0.013297   (1.4064730974135704, 0.276598752507528)   0.340495   
 3  bm1_mkt4  -0.011120  (1.027599635071022, 0.12098413566267148)   0.212150   
 4  bm1_mkt5  -0.012110  (0.38076027968612, -0.04741765379273745)   0.041799   
 
                                          p_values  
 0  (1.0713150996396288e-17, 0.058534118703877834)  
 1   (1.3735206622717943e-14, 0.1

In [140]:
table1_df.to_excel('CAPM.xls')
table2_df.to_excel('twofactors.xls')
table3_df.to_excel('FAMA.xls')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_10140\836913268.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  table1_df.to_excel('CAPM.xls')
C:\Users\lenovo\AppData\Local\Temp\ipykernel_10140\836913268.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, 